# GTFS and AVL Summary Tables

In [ ]:
import pandas as pd
import os
import numpy as np
# Set the workspace environment to master folder
x=os.chdir('C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch')

## GTFS - gives start/end time for each trip, each day (no bus info)

In [ ]:
#open the outputs from GTFS Joining(PANDAS)
calendar = pd.read_csv('Data\\GTFS_Query\\cal_select.txt', index_col = False)
routes = pd.read_csv('Data\\GTFS_Query\\routes_select.txt', index_col = False)
shapes = pd.read_csv('Data\\GTFS_Query\\shapes_select.txt', index_col = False)
stop_times= pd.read_csv('Data\\GTFS_Query\\stop_times_select.txt', index_col = False)
stops = pd.read_csv('Data\\GTFS_Query\\stops_select.txt', index_col = False)
trips = pd.read_csv('Data\\GTFS_Query\\trips_select.txt', index_col = False)

In [ ]:
#should give you stop times w/ stop locations for each trip
add_trip_stop_times = pd.merge(stop_times,trips,how = 'left', on = ['trip_id','route_id','service_id', 'trip_headsign','shape_id'])

In [ ]:
#error happens here? Fixed: left to outer join
add_trip_stops = pd.merge(add_trip_stop_times,stops, how = 'outer', on = ['stop_id'])
#add_trip_stops.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\ProcessedAVL_GTFS\Test.csv", index = False)

add_trip_stops = add_trip_stops.sort_values(by =['trip_id', 'stop_sequence_x'])
add_trip_stops.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\ProcessedAVL_GTFS\Test.csv", index = False)
add_trip_stops

In [ ]:
add_trip_stops['SERV_DAY'] = add_trip_stops['trip_id'].str[22:-3]
for index in range(len(add_trip_stops)):
    if 'Northbound' in add_trip_stops.loc[index,'trip_headsign']:
        add_trip_stops.loc[index,'dir_sign'] = 4
    if 'Southbound' in add_trip_stops.loc[index,'trip_headsign']:
        add_trip_stops.loc[index,'dir_sign'] = 1
GTFSSum = add_trip_stops.replace({np.nan: None})
#GTFSSum.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\ProcessedAVL_GTFS\Test.csv", index = False)

In [ ]:
groupedtrips = GTFSSum.groupby("trip_id",sort=True)
for group in groupedtrips:
    a = groupedtrips.head(1)
    b = groupedtrips.tail(1)

In [ ]:
final = pd.merge(a,b, on= 'trip_id',suffixes = ('_start','_end'))
sumtableGTFS = final[['trip_id',
                     'service_id_start',
                     'trip_headsign_start',
                      'dir_sign_start',
                      'SERV_DAY_start',
                     
                     'stop_lat_start',
                      'stop_lon_start',
                      'arrival_time_start',
                      'departure_time_start',
                     'stop_id_start',
                     'stop_name_start',
                     'stop_desc_start',
                     
                      'stop_lat_end',
                      'stop_lon_end',
                      'arrival_time_end',
                      'departure_time_end',
                     'stop_id_end',
                     'stop_name_end',
                     'stop_desc_end'
                     ]]
sumtableGTFS= sumtableGTFS.sort_values(['service_id_start', 'dir_sign_start','arrival_time_start'])
sumtableGTFS.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\ProcessedAVL_GTFS\GTFS_Trips.csv", index = False)
#nice, in service ID, direction, trio arrangement

## AVL - proof table

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, tzinfo
from dateutil import tz
import pytz
dataPath = "C:\\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Aline-vehicle-messages-Oct1_Oct8_2016\Aline-vehicle-messages-Oct1_Oct8-2016.csv"
dataFrame = pd.read_csv(dataPath)
avl_messages = dataFrame[["CALENDAR_ID",
                          "MESSAGE_TIMESTAMP",
                          "LOCAL_TIMESTAMP",
                          "SOURCE_HOST",
                          "ROUTE_ABBR",
                          "DIRECTION",
                          "LONGITUDE",
                          "LATITUDE"]]
newframe = avl_messages.replace({np.nan: None})
newframe

In [ ]:
#Fix local_time for each row in AVL full data 
for index in range(len(newframe)):
    try:
        dt_str = newframe.loc[index, 'LOCAL_TIMESTAMP']
        dt_str1 = dt_str.replace('T',"").replace('Z',"")
        t3 = datetime.strptime(dt_str1,"%Y-%m-%d%H:%M:%S.%f")
        newframe.loc[index,'TIME'] = t3
    except:
        pass

Local timestamp is not as specific as message timestamp;
several messages in a row are given the same time.
But say we remove any duplicates,
so there is just one of each local timestamp time for simplicity 

In [ ]:
#will eventually drop this cell
#works for both versions
newframe = newframe.drop_duplicates(subset = ['TIME'], keep = 'last')

In [ ]:
#sorts by bus # and then stripped LOCAL_TIME or ('TIME')
time_sorted = newframe.sort_values(["SOURCE_HOST",
                                        'TIME'],
                                       axis=0,
                                       ascending=(True, True))

In [ ]:
#group by source host (keeps them in time order) and reset index
groupedhost = time_sorted.groupby('SOURCE_HOST')
time_sorted.reset_index(drop=True)

#Mark rows where the day ends and new day begins
time_sorted['PREVDAY'] = groupedhost.transform(lambda x: x.shift(-1) > x+pd.Timedelta('1h'))
time_sorted['NEXTDAY'] = time_sorted['PREVDAY'].shift(1)
time_sorted.reset_index(drop = True)

In [ ]:
#Select rows with marks, put them together, final sort, export
final = time_sorted.loc[time_sorted['PREVDAY'] == True] 
final2 = time_sorted.loc[time_sorted['NEXTDAY'] == True]
x = pd.concat([final,final2])
y = x.sort_values(["SOURCE_HOST",'TIME'],axis=0,ascending=(True, True))
y.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\ProcessedAVL_GTFS\AVL_StartStop_ofDay_EachBus.csv",  index=False)

AVL: Grouped by bus, then time, then takes first and last time for each service day <br>
GTFS: Sorted by service day, then direction, assign trip numbers (+1 when direction change, reset at day change), start/stop each trip *Good to go*

So we'll still need an AVL that is service_day, then time, assign trip numbers (+1 when direction change, reset at day change). For each day, line up GTFS and AVL against each other to compare

# AVL Trips (to match GTFS)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, tzinfo
from dateutil import tz
import pytz
dataPath = "C:\\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Aline-vehicle-messages-Oct1_Oct8_2016\Aline-vehicle-messages-Oct1_Oct8-2016.csv"
dataFrame = pd.read_csv(dataPath)
avl_messages = dataFrame[["CALENDAR_ID",
                          "MESSAGE_TIMESTAMP",
                          "LOCAL_TIMESTAMP",
                          "SOURCE_HOST",
                          "ROUTE_ABBR",
                          "DIRECTION",
                          "LONGITUDE",
                          "LATITUDE"]]
newframe = avl_messages.replace({np.nan: None})

In [ ]:
#Fix local_time for each row in AVL full data 
for index in range(len(newframe)):
    try:
        dt_str = newframe.loc[index, 'MESSAGE_TIMESTAMP']
        dt_str1 = dt_str.replace('T',"").replace('Z',"")
        dt_utc = datetime.strptime(dt_str1,"%Y-%m-%d%H:%M:%S.%f")
        dt_utc = dt_utc.replace(tzinfo = pytz.UTC)
        local_zone = tz.tzlocal()
        dt_local = dt_utc.astimezone(local_zone)
        local_time_str = dt_local.strftime("%Y-%m-%d%H:%M:%S.%f")
        print(local_time_str)
        
        
        
        newframe.loc[index,'TIME'] = local_time_str
    except:
        pass

In [ ]:
#something like give value until >3 hr time gap, then add one